In [40]:
# Set the working directory
import os
os.getcwd()
os.chdir('/home/shoyo/Desktop')

In [41]:
# Import, authenticate and initialize the Earth Engine library
import ee
ee.Initialize()

# Import modules
import pandas as pd
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import *
!pip install geopandas --quiet
import geopandas as gpd
import numpy as np
# !pip install datatable --quiet
!pip install git+https://github.com/h2oai/datatable --quiet
# !pip install datatable --quiet
import datatable as dt
from datatable import f
import time
from pprint import pprint
!pip install gee_subset --quiet
from gee_subset import gee_subset
import shapely.geometry
import re
!pip install geemap --quiet
import geemap
!pip install git+https://github.com/loicdtx/landsat-extract-gee --quiet
# !pip install geextract
import geextract
from geextract import ts_extract
import math
import rasterio
from rasterio.plot import show
import glob
import random

ERROR: Command errored out with exit status 128: git clone -q https://github.com/h2oai/datatable /tmp/pip-req-build-8suiklb1 Check the logs for full command output.
ERROR: Command errored out with exit status 128: git clone -q https://github.com/loicdtx/landsat-extract-gee /tmp/pip-req-build-7bvj88x8 Check the logs for full command output.


In [42]:
# Load the SOC ground data
csv_path = '/SOC_LonLatYearLCSite.csv'
df = pd.read_csv(os.getcwd() + csv_path)
df = df.sample(frac=1, random_state=42)
df

,longitude,latitude,year,LC,siteid,site,SOC
3476,31.329419,-26.283403,2020,grassland,ludzeludze_4_5,ludzeludze,18.08215
463,37.557439,0.718338,2021,bushland,kalama_7_3,kalama,8.97930
3857,104.344126,21.508767,2020,cropland,yenbai_3_10,yenbai,10.38540
3173,37.598133,0.664169,2021,shrubland,kalama_13_10,kalama,4.13980
2092,32.978996,-3.913196,2021,cropland,sigili_8_3,sigili,3.63550
...,...,...,...,...,...,...,...
1130,39.063375,-0.089876,2022,shrubland,mbalambala_16_8,mbalambala,8.04395
1294,37.680868,-2.651153,2022,grassland,mbirikani_10_6,mbirikani,7.54800
860,34.812098,2.740625,2021,wooded_grassland,lokiriama_4_3,lokiriama,7.69240
3507,31.344780,-26.279360,2020,freshwater_aquatic,ludzeludze_8_10,ludzeludze,12.93300


In [43]:
# Set seed
def set_seed(seed: int = 42) -> None:
  random.seed(seed)
  np.random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

In [44]:
# Functions to mask clouds, add EVI and obtain Landsat data at the specified location

def cloudMask(image):
  qa = image.select('QA_PIXEL')
  # dilated = 1 << 1, cirrus = 1 << 2, cloud = 1 << 3, shadow = 1 << 4
  mask = ((qa.bitwise_and(1 << 3).neq(0)).multiply(1)
      .add((qa.bitwise_and(1 << 4).neq(0)).multiply(1))
      .add((qa.bitwise_and(1 << 2).neq(0)).multiply(1))
      # .add((qa.bitwise_and(1 << 1).neq(0)).multiply(1))
      )
  return image.updateMask(mask)

def addEVI(image):
  SR = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  image = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  image = image.addBands(SR, None, True)
  evi = image.expression( '2.5 * ( (NIR-RED) / (NIR + 6*RED - 7.5*BLUE + 1))', {
      'NIR': image.select('SR_B5'),
      'RED': image.select('SR_B4'),
      'BLUE': image.select('SR_B2'),
      }).rename('EVI')
  return image.addBands(evi)

def get_landsat(lon, lat, start, end):
  point = ee.Geometry.Point(lon, lat).buffer(15).bounds()
  landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start, end).filterBounds(point).map(cloudMask).map(addEVI).select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI'])
  landsat9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterDate(start, end).filterBounds(point).map(cloudMask).map(addEVI).select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI'])
  landsat = landsat8.merge(landsat9).median()
  landsat = ee.ImageCollection.fromImages([landsat]).getRegion(point, 30).getInfo()
  return landsat

In [45]:
# Functions for Sentinel-1 preprocessing

# Linear to db scale
def lin_to_db(image):
    bandNames = image.bandNames().remove('angle')
    db = ee.Image.constant(10).multiply(image.select(bandNames).log10()).rename(bandNames)
    return image.addBands(db, None, True)

def db_to_lin(image):
    bandNames = image.bandNames().remove('angle')
    lin = ee.Image.constant(10).pow(image.select(bandNames).divide(10)).rename(bandNames)
    return image.addBands(lin, None, True)

def lin_to_db2(image):
    db = ee.Image.constant(10).multiply(image.select(['VV', 'VH']).log10()).rename(['VV', 'VH'])
    return image.addBands(db, None, True)

# Add ratio bands
def add_ratio_lin(image):
    ratio = image.addBands(image.select('VV').divide(image.select('VH')).rename('VVVH_ratio'))
    
    return ratio.set('system:time_start', image.get('system:time_start'))

# Additional Border Noise Removal
def maskAngLT452(image):
    ang = image.select(['angle'])
    return image.updateMask(ang.lt(45.23993)).set('system:time_start', image.get('system:time_start'))

def maskAngGT30(image):
    ang = image.select(['angle'])
    return image.updateMask(ang.gt(30.63993)).set('system:time_start', image.get('system:time_start'))

def maskEdge(image):
    mask = image.select(0).unitScale(-25, 5).multiply(255).toByte()#.connectedComponents(ee.Kernel.rectangle(1,1), 100)
    return image.updateMask(mask.select(0)).set('system:time_start', image.get('system:time_start')) 

def f_mask_edges(image):
    db_img = lin_to_db(image)
    output = maskAngGT30(db_img)
    output = maskAngLT452(output)
    #output = maskEdge(output)
    output = db_to_lin(output)
    return output.set('system:time_start', image.get('system:time_start'))

# 1.SPECKLE FILTERS
def boxcar(image, KERNEL_SIZE):
    bandNames = image.bandNames().remove('angle')
    #Define a boxcar kernel
    kernel = ee.Kernel.square((KERNEL_SIZE/2), units='pixels', normalize=True)
    #Apply boxcar
    output = image.select(bandNames).convolve(kernel).rename(bandNames)
    return image.addBands(output, None, True)

def RefinedLee(image):
    """
    This filter is modified from the implementation by Guido Lemoine 
    Source: Lemoine et al. https://code.earthengine.google.com/5d1ed0a0f0417f098fdfd2fa137c3d0c
    """
    bandNames = image.bandNames().remove('angle')
    def inner(b):
        img = image.select([b]);
        # img must be linear, i.e. not in dB!
        # Set up 3x3 kernels 
        weights3 = ee.List.repeat(ee.List.repeat(1,3),3);
        kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False);
        mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3);
        variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3);

        # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
        sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]]);
  
        sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False);
  
        # Calculate mean and variance for the sampled windows and store as 9 bands
        sample_mean = mean3.neighborhoodToBands(sample_kernel); 
        sample_var = variance3.neighborhoodToBands(sample_kernel);
  
        # Determine the 4 gradients for the sampled windows
        gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs();
        gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs());
        gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs());
        gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs());
  
        # And find the maximum gradient amongst gradient bands
        max_gradient = gradients.reduce(ee.Reducer.max());
  
        # Create a mask for band pixels that are the maximum gradient
        gradmask = gradients.eq(max_gradient);
  
        # duplicate gradmask bands: each gradient represents 2 directions
        gradmask = gradmask.addBands(gradmask);
  
        # Determine the 8 directions
        directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1);
        directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2));
        directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3));
        directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4));
        # The next 4 are the not() of the previous 4
        directions = directions.addBands(directions.select(0).Not().multiply(5));
        directions = directions.addBands(directions.select(1).Not().multiply(6));
        directions = directions.addBands(directions.select(2).Not().multiply(7));
        directions = directions.addBands(directions.select(3).Not().multiply(8));
  
        # Mask all values that are not 1-8
        directions = directions.updateMask(gradmask);
  
        # "collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
        directions = directions.reduce(ee.Reducer.sum());  
  
        sample_stats = sample_var.divide(sample_mean.multiply(sample_mean));
  
        #Calculate localNoiseVariance
        sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0]);
  
        # Set up the 7*7 kernels for directional statistics
        rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4));
  
        diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]]);
  
        rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False);
        diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False);
  
        # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
        dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1));
        dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1));
  
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)));
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)));
  
        # and add the bands for rotated kernels
        for i in range(1, 4):
            dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
            dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
            dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
            dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))

  
        # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
        dir_mean = dir_mean.reduce(ee.Reducer.sum());
        dir_var = dir_var.reduce(ee.Reducer.sum());
  
        # A finally generate the filtered value
        varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))
  
        b = varX.divide(dir_var)
        result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
  
        return result.arrayProject([0]).arrayFlatten([['sum']]).float()
    
    result = ee.ImageCollection(bandNames.map(inner)).toBands().rename(bandNames).copyProperties(image)
    
    return image.addBands(result, None, True) 


# 2. MONO-TEMPORAL SPECKLE FILTER (WRAPPER)
def MonoTemporal_Filter(coll,KERNEL_SIZE, SPECKLE_FILTER) :
    def _filter(image):    
       if (SPECKLE_FILTER=='REFINED LEE'):
          _filtered = RefinedLee(image)
    return coll.map(_filter)


# 3. MULTI-TEMPORAL SPECKLE FILTER
def MultiTemporal_Filter(coll,KERNEL_SIZE, SPECKLE_FILTER,NR_OF_IMAGES):
    def Quegan(image) :
        """
        The following Multi-temporal speckle filters are implemented as described in
        S. Quegan and J. J. Yu, “Filtering of multichannel SAR images,” 
        IEEE Trans Geosci. Remote Sensing, vol. 39, Nov. 2001.
        """
        def setresample(image):
                return image.resample()
            
        def get_filtered_collection(image):  
            #filter collection over are and by relative orbit
            s1_coll = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
                .filterBounds(image.geometry()) \
                .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', ee.List(image.get('transmitterReceiverPolarisation')).get(-1))) \
                .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_stop')), \
                                     ee.Filter.eq('relativeOrbitNumber_stop', image.get('relativeOrbitNumber_start'))
                )).map(setresample)
      
            #a function that takes the image and checks for the overlap
            def check_overlap(_image):
                # get all S1 frames from this date intersecting with the image bounds
                s1 = s1_coll.filterDate(_image.date(), _image.date().advance(1, 'day'))
                # intersect those images with the image to filter
                intersect = image.geometry().intersection(s1.geometry().dissolve(), 10)
                # check if intersect is sufficient
                valid_date = ee.Algorithms.If(intersect.area(10).divide(image.geometry().area(10)).gt(0.95), \
                                              _image.date().format('YYYY-MM-dd')
                                              )
                return ee.Feature(None, {'date': valid_date})
      
      
            # this function will pick up the acq dates for fully overlapping acquisitions before the image acquistion
            dates_before = s1_coll.filterDate('2014-01-01', image.date().advance(1, 'day')) \
                                    .sort('system:time_start', False).limit(5*NR_OF_IMAGES) \
                                    .map(check_overlap).distinct('date').aggregate_array('date')
    
            # if the images before are not enough, we add images from after the image acquisition 
            # this will only be the case at the beginning of S1 mission
            dates = ee.List(ee.Algorithms.If( \
                                             dates_before.size().gte(NR_OF_IMAGES), \
                                                 dates_before.slice(0, NR_OF_IMAGES), \
                                                     s1_coll \
                                                         .filterDate(image.date(), '2100-01-01') \
                                                             .sort('system:time_start', True).limit(5*NR_OF_IMAGES) \
                                                                 .map(check_overlap) \
                                                                     .distinct('date') \
                                                                         .aggregate_array('date') \
                                                                             .cat(dates_before).distinct().sort().slice(0, NR_OF_IMAGES)
                                                                             )
                                                )
    
            #now we re-filter the collection to get the right acquisitions for multi-temporal filtering
            return ee.ImageCollection(dates.map(lambda date: s1_coll.filterDate(date, ee.Date(date).advance(1,'day')).toList(s1_coll.size())).flatten())
      
          
  
        #we get our dedicated image collection for that image
        s1 = get_filtered_collection(image)
  
        bands = image.bandNames().remove('angle')
        s1 = s1.select(bands)
        meanBands = bands.map(lambda bandName: ee.String(bandName).cat('_mean'))
        ratioBands = bands.map(lambda bandName: ee.String(bandName).cat('_ratio'))
        count_img = s1.reduce(ee.Reducer.count())

        def inner(image):
            # if (SPECKLE_FILTER=='BOXCAR'):
            #     _filtered = boxcar(image, KERNEL_SIZE).select(bands).rename(meanBands) 
            # elif (SPECKLE_FILTER=='LEE'):
            #     _filtered = leefilter(image, KERNEL_SIZE).select(bands).rename(meanBands)
            # elif (SPECKLE_FILTER=='GAMMA MAP'):
            #     _filtered = gammamap(image, KERNEL_SIZE).select(bands).rename(meanBands)
            if (SPECKLE_FILTER=='REFINED LEE'):
                _filtered = RefinedLee(image).select(bands).rename(meanBands)
            # elif (SPECKLE_FILTER=='LEE SIGMA'):
            #     _filtered = leesigma(image, KERNEL_SIZE).select(bands).rename(meanBands)
    
            _ratio = image.select(bands).divide(_filtered).rename(ratioBands) 
            return _filtered.addBands(_ratio)

        isum = s1.map(inner).select(ratioBands).reduce(ee.Reducer.sum())
        filtered = inner(image).select(meanBands)
        divide = filtered.divide(count_img)
        output = divide.multiply(isum).rename(bands)

        return image.addBands(output, None, True)
    return coll.map(Quegan)

# Terrain Flattening
def slope_correction(collection, TERRAIN_FLATTENING_MODEL, DEM, TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER):
    ninetyRad = ee.Image.constant(90).multiply(math.pi/180)

    def _volumetric_model_SCF(theta_iRad, alpha_rRad):
        # Volume model
        nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
        denominator = (ninetyRad.subtract(theta_iRad)).tan()
        return nominator.divide(denominator)

    def _direct_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
        # Surface model
        nominator = (ninetyRad.subtract(theta_iRad)).cos()
        denominator = alpha_azRad.cos().multiply((ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos())
        return nominator.divide(denominator)

    def _erode(image, distance):
        # buffer function (thanks Noel)
        d = (image.Not().unmask(1).fastDistanceTransform(30).sqrt()
             .multiply(ee.Image.pixelArea().sqrt()))
        return image.updateMask(d.gt(distance))

    def _masking(alpha_rRad, theta_iRad, buffer):
        # calculate masks
        # layover, where slope > radar viewing angle
        layover = alpha_rRad.lt(theta_iRad).rename('layover')
        # shadow
        shadow = alpha_rRad.gt(ee.Image.constant(-1)
                        .multiply(ninetyRad.subtract(theta_iRad))).rename('shadow')
        # combine layover and shadow
        mask = layover.And(shadow)
        # add buffer to final mask
        if (buffer > 0):
            mask = _erode(mask, buffer)
        return mask.rename('no_data_mask')

    def _correct(image):
        bandNames = image.bandNames()
        geom = image.geometry()
        proj = image.select(1).projection()
        elevation = DEM.resample('bilinear').reproject(proj,None, 10).clip(geom)
        # calculate the look direction
        heading = ee.Terrain.aspect(image.select('angle')).reduceRegion(ee.Reducer.mean(), image.geometry(), 1000)
        
        #in case of null values for heading replace with 0
        heading = ee.Dictionary(heading).combine({'aspect': 0}, False).get('aspect')
        
        heading = ee.Algorithms.If(
            ee.Number(heading).gt(180),
            ee.Number(heading).subtract(360),
            ee.Number(heading)
        )
        
        # the numbering follows the article chapters
        # 2.1.1 Radar geometry
        theta_iRad = image.select('angle').multiply(math.pi/180)
        phi_iRad = ee.Image.constant(heading).multiply(math.pi/180)
        
        # 2.1.2 Terrain geometry
        alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(math.pi / 180)

        aspect = ee.Terrain.aspect(elevation).select('aspect').clip(geom)
        
        aspect_minus = aspect.updateMask(aspect.gt(180)).subtract(360)
        
        phi_sRad = aspect.updateMask(aspect.lte(180))\
            .unmask()\
            .add(aspect_minus.unmask())\
            .multiply(-1)\
            .multiply(math.pi / 180)
          
        #elevation = DEM.reproject(proj,None, 10).clip(geom)

        # 2.1.3 Model geometry
        # reduce to 3 angle
        phi_rRad = phi_iRad.subtract(phi_sRad)

        # slope steepness in range (eq. 2)
        alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

        # slope steepness in azimuth (eq 3)
        alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

        # 2.2
        # Gamma_nought
        gamma0 = image.divide(theta_iRad.cos())

        if (TERRAIN_FLATTENING_MODEL == 'VOLUME'):
            # Volumetric Model
            scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

        if (TERRAIN_FLATTENING_MODEL == 'DIRECT'):
            scf = _direct_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

        # apply model for Gamm0
        gamma0_flat = gamma0.multiply(scf)

        # get Layover/Shadow mask
        mask = _masking(alpha_rRad, theta_iRad, TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER)
        output = gamma0_flat.mask(mask).rename(bandNames).copyProperties(image)
        output = ee.Image(output).addBands(image.select('angle'), None, True)

        return output.set('system:time_start', image.get('system:time_start'))
    return collection.map(_correct)

def s1_preproc(params):
    APPLY_BORDER_NOISE_CORRECTION = params['APPLY_BORDER_NOISE_CORRECTION']
    APPLY_TERRAIN_FLATTENING = params['APPLY_TERRAIN_FLATTENING']
    APPLY_SPECKLE_FILTERING = params['APPLY_SPECKLE_FILTERING']
    POLARIZATION = params['POLARIZATION']
    # PLATFORM_NUMBER = params['PLATFORM_NUMBER']
    ORBIT = params['ORBIT']
    # ORBIT_NUM = params['ORBIT_NUM']
    SPECKLE_FILTER_FRAMEWORK = params['SPECKLE_FILTER_FRAMEWORK']
    SPECKLE_FILTER = params['SPECKLE_FILTER']
    SPECKLE_FILTER_KERNEL_SIZE = params['SPECKLE_FILTER_KERNEL_SIZE']
    SPECKLE_FILTER_NR_OF_IMAGES = params['SPECKLE_FILTER_NR_OF_IMAGES']
    TERRAIN_FLATTENING_MODEL = params['TERRAIN_FLATTENING_MODEL']
    DEM = params['DEM']
    TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER = params['TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER']
    FORMAT = params['FORMAT']
    START_DATE = params['START_DATE']
    STOP_DATE = params['STOP_DATE']
    ROI = params['ROI']
    CLIP_TO_ROI = params['CLIP_TO_ROI']
    SAVE_ASSET = params['SAVE_ASSET']
    # ASSET_ID = params['ASSET_ID']

    # 0. CHECK PARAMETERS
    if APPLY_BORDER_NOISE_CORRECTION is None:
        APPLY_BORDER_NOISE_CORRECTION = True
    if APPLY_TERRAIN_FLATTENING is None:
        APPLY_TERRAIN_FLATTENING = True
    if APPLY_SPECKLE_FILTERING is None:
        APPLY_SPECKLE_FILTERING = True
    if POLARIZATION is None:
        POLARIZATION = 'VVVH'
    if ORBIT is None:
        ORBIT = 'BOTH'
    if SPECKLE_FILTER_FRAMEWORK is None:
        SPECKLE_FILTER_FRAMEWORK = 'MULTI'
    if SPECKLE_FILTER is None:
        SPECKLE_FILTER = 'REFINED LEE'
    if SPECKLE_FILTER_KERNEL_SIZE is None:
        SPECKLE_FILTER_KERNEL_SIZE = 7
    if SPECKLE_FILTER_NR_OF_IMAGES is None:
        SPECKLE_FILTER_NR_OF_IMAGES = 10
    if TERRAIN_FLATTENING_MODEL is None:
        TERRAIN_FLATTENING_MODEL = 'VOLUME'
    if TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER is None:
        TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER = 0
    if FORMAT is None:
        FORMAT = 'DB'
    # if ORBIT is None:
    #     ORBIT = 'DESCENDING'

    pol_required = ['VV', 'VH', 'VVVH']
    if (POLARIZATION not in pol_required):
        raise ValueError("ERROR!!! Parameter POLARIZATION not correctly defined")

    orbit_required = ['ASCENDING', 'DESCENDING', 'BOTH']
    if (ORBIT not in orbit_required):
        raise ValueError("ERROR!!! Parameter ORBIT not correctly defined")

    model_required = ['DIRECT', 'VOLUME']
    if (TERRAIN_FLATTENING_MODEL not in model_required):
        raise ValueError("ERROR!!! Parameter TERRAIN_FLATTENING_MODEL not correctly defined")

    format_required = ['LINEAR', 'DB']
    if (FORMAT not in format_required):
        raise ValueError("ERROR!!! FORMAT not correctly defined")

    frame_needed = ['MONO', 'MULTI']
    if (SPECKLE_FILTER_FRAMEWORK not in frame_needed):
        raise ValueError("ERROR!!! SPECKLE_FILTER_FRAMEWORK not correctly defined")

    format_sfilter = ['BOXCAR', 'LEE', 'GAMMA MAP'
              ,'REFINED LEE', 'LEE SIGMA']
    
    if (SPECKLE_FILTER not in format_sfilter):
        raise ValueError("ERROR!!! SPECKLE_FILTER not correctly defined")

    if (TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER < 0):
        raise ValueError("ERROR!!! TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER not correctly defined")

    if (SPECKLE_FILTER_KERNEL_SIZE <= 0):
        raise ValueError("ERROR!!! SPECKLE_FILTER_KERNEL_SIZE not correctly defined")


    # 1. DATA SELECTION
    # select S-1 image collection
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')\
        .filter(ee.Filter.eq('instrumentMode', 'IW'))\
        .filter(ee.Filter.eq('resolution_meters', 10)) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
        .filterDate(START_DATE, STOP_DATE) \
        .filterBounds(ROI)

    # if (PLATFORM_NUMBER=='A' or PLATFORM_NUMBER=='B' ):
    #     s1 = s1.filter(ee.Filter.eq('platform_number', PLATFORM_NUMBER))
   

    # if (ORBIT_NUM != None):
    #     s1 = s1.filter(ee.Filter.eq('relativeOrbitNumber_start',ORBIT_NUM))
    #     #.filter(ee.Filter.eq('relativeOrbitNumber_start',None))


    # select orbit
    if (ORBIT != 'BOTH'):
        s1 = s1.filter(ee.Filter.eq('orbitProperties_pass', ORBIT))

    # select polarization
    if (POLARIZATION == 'VV'):
        s1 = s1.select(['VV', 'angle'])
    elif (POLARIZATION == 'VH'):
        s1 = s1.select(['VH', 'angle'])
    elif (POLARIZATION == 'VVVH'):
        s1 = s1.select(['VV', 'VH', 'angle'])
        
    # print('Number of images in collection: ', s1.size().getInfo())

    # 2. ADDITIONAL BORDER NOISE CORRECTION
    if (APPLY_BORDER_NOISE_CORRECTION):
        s1_1 = s1.map(f_mask_edges)
        # print('Additional border noise correction is completed')
    else:
        s1_1 = s1

    # 3. SPECKLE FILTERING
    if (APPLY_SPECKLE_FILTERING):
        if (SPECKLE_FILTER_FRAMEWORK == 'MONO'):
            s1_1 = ee.ImageCollection(MonoTemporal_Filter(s1_1, SPECKLE_FILTER_KERNEL_SIZE, SPECKLE_FILTER))
            # print('Mono-temporal speckle filtering is completed')
        else:
            s1_1 = ee.ImageCollection(MultiTemporal_Filter(s1_1, SPECKLE_FILTER_KERNEL_SIZE, SPECKLE_FILTER, SPECKLE_FILTER_NR_OF_IMAGES))
            # print('Multi-temporal speckle filtering is completed')

    # 4. TERRAIN CORRECTION
    if (APPLY_TERRAIN_FLATTENING):
        s1_1 = (slope_correction(s1_1, TERRAIN_FLATTENING_MODEL ,DEM, TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER))
        # print('Radiometric terrain normalization is completed')

    # 5. OUTPUT
    if (FORMAT == 'DB'):
        s1_1 = s1_1.map(lin_to_db)
              
    #clip to roi
    if (CLIP_TO_ROI):
        s1_1 = s1_1.map(lambda image: image.clip(ROI))
    
    # print( type(s1_1) )
    return s1_1.median()

# Function to obtain Sentinel-1 backscatter value at the specified location
def getSAR(lon, lat, start, end):
    parameter = {'START_DATE': start,
                'STOP_DATE': end,        
                'POLARIZATION': 'VVVH',
                'ORBIT' : 'BOTH',
                'ROI': ee.Geometry.Point(lon, lat).buffer(15).bounds(),
                'APPLY_BORDER_NOISE_CORRECTION': True,
                'APPLY_SPECKLE_FILTERING': True,
                'SPECKLE_FILTER_FRAMEWORK': 'MULTI',
                'SPECKLE_FILTER': 'REFINED LEE',
                'SPECKLE_FILTER_KERNEL_SIZE': 9,
                'SPECKLE_FILTER_NR_OF_IMAGES':10,
                'APPLY_TERRAIN_FLATTENING': True,
                'DEM': ee.Image('USGS/SRTMGL1_003'),
                'TERRAIN_FLATTENING_MODEL': 'VOLUME',
                'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
                'FORMAT': 'LINEAR',
                'CLIP_TO_ROI': False,
                'SAVE_ASSET': False, 
                }
    #processed s1 collection
    s1_processed = s1_preproc(parameter)
    s1_processed = ee.ImageCollection.fromImages([s1_processed]).getRegion(parameter['ROI'], 10).getInfo()

    return s1_processed

In [20]:
# Functions to obtain elevation, topography and climate data at the specified location
def getElev(lon, lat):
  point = ee.Geometry.Point(lon, lat).buffer(15).bounds()
  elev = ee.Image("USGS/SRTMGL1_003").select(['elevation'])
  elevation = ee.ImageCollection.fromImages([elev]).getRegion(point, 30).getInfo()
  slop = ee.Terrain.slope(elev)
  slope = ee.ImageCollection.fromImages([slop]).getRegion(point, 30).getInfo()
  return elevation, slope

def getTopo(lon, lat):
  point = ee.Geometry.Point(lon, lat).buffer(45).bounds()
  aspect = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/aspect").getRegion(point, 90).getInfo()
  roughness = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/roughness").getRegion(point, 90).getInfo()
  twi = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/cti").getRegion(point, 90).getInfo()
  return aspect, roughness, twi

def getPrecip(lon, lat, start, end):
  point = ee.Geometry.Point(lon, lat).buffer(15).bounds()
  rain = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").select(['precipitation']).filterDate(start, end).filterBounds(point).median()
  temp = ee.ImageCollection("MODIS/061/MOD11A1").select(['LST_Day_1km']).filterDate(start, end).filterBounds(point).median()
  precipitation = ee.ImageCollection.fromImages([rain]).getRegion(point, 30).getInfo()
  temperature = ee.ImageCollection.fromImages([temp]).getRegion(point, 30).getInfo()
  return precipitation, temperature

# Check if the function works
# getElev(37.9062, 0.0236)
# getTopo(37.9062, 0.0236)
# getPrecip(37.9062, 0.0236, ee.Date('2021-01-01'), ee.Date('2021-12-31'))

In [ ]:
# Extract Landsat, Sentinel-1, topography and climate data at the each site

dataframe = pd.DataFrame(columns=['longitude', 'latitude', 'year', 'LC', 'siteid', 'site', 'SOC', 
                                  'elev', 'slope', 'aspect', 'roughness', 'twi', 
                                  'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI', 'VV', 'VH', 'VV/VH', 'prec', 'temp'
                                  ])

for i in range(len(df)):
  print(i)

  elev, slope = getElev(df.iloc[i, 0], df.iloc[i, 1])[0], getElev(df.iloc[i, 0], df.iloc[i, 1])[1]
  aspect, roughness, twi = getTopo(df.iloc[i, 0], df.iloc[i, 1])[0], getTopo(df.iloc[i, 0], df.iloc[i, 1])[1], getTopo(df.iloc[i, 0], df.iloc[i, 1])[2]
  newCol = pd.Series( df.iloc[i, :7].to_list() + [ elev[1][4], slope[1][4], aspect[1][4], roughness[1][4], twi[1][4] ], index=dataframe.columns[:12] )
  
  landsat = get_landsat(df.iloc[i, 0], df.iloc[i, 1], 
                     ee.Date(str(df.iloc[i, 2]) + '-01-01'), 
                     ee.Date(str(df.iloc[i, 2]) + '-12-31')
                     )
  sar = getSAR(df.iloc[i, 0], df.iloc[i, 1], 
                     ee.Date(str(df.iloc[i, 2]) + '-01-01'), 
                     ee.Date(str(df.iloc[i, 2]) + '-12-31')
                     )
    
  precipitation, temperature = getPrecip(df.iloc[i, 0], df.iloc[i, 1], 
                     ee.Date(str(df.iloc[i, 2]) + '-01-01'), 
                     ee.Date(str(df.iloc[i, 2]) + '-12-31')
                     )[0], getPrecip(df.iloc[i, 0], df.iloc[i, 1], 
                     ee.Date(str(df.iloc[i, 2]) + '-01-01'), 
                     ee.Date(str(df.iloc[i, 2]) + '-12-31')
                     )[1]                      
        
  addCol = pd.Series( [ landsat[1][l] for l in range(4, 11) ] 
                     + [ sar[1][l] for l in range(4, 6) ] + [ sar[1][4]/sar[1][5] ]
                     + [ precipitation[1][4], temperature[1][4] ],
                     index=dataframe.columns[12:24] 
                     )
  # newCol = newCol.append(addCol)
  newCol = pd.concat([newCol, addCol])
  # print('newCol', newCol)

  if newCol.isna().any() == True:
    print(newCol)
    
  # dataframe = dataframe.append(newRow, ignore_index=True)
  dataframe.loc[i] = newCol
  # print(dataframe)

dataframe

In [36]:
# DataFrame of rows containing no data
nodata = dataframe[dataframe.isna().any(axis=1)].loc[:, ['latitude','longitude']]
nodata

,latitude,longitude


In [ ]:
# Remove nodata from the whole tabular data
dftrue = pd.DataFrame(columns=dataframe.columns)
for i in range(len(dataframe)):
    lat = dataframe.loc[i, 'latitude']
    lon = dataframe.loc[i, 'longitude']
    if not ((nodata['latitude'] == lat) & (nodata['longitude'] == lon)).any():
        # dftrue = dftrue.append(dataframe.loc[i])
        dftrue.loc[i] = dataframe.loc[i]
dftrue[dftrue.isna().any(axis=1)]

In [70]:
dftrue = dftrue * 0.02
dftrue

,longitude,latitude,year,SOC,LC,elev,slope,aspect,roughness,twi,...,SR_B5,SR_B6,SR_B7,EVI,ST_B10,VV,VH,VV/VH,prec,temp
0,35.191816,1.37833,2021,6.84225,cropland,1641,1.311631,47.303562,15.660496,-0.83046,...,0.421858,0.312834,0.240069,0.20156,278.537831,0.100637,0.016779,5.997724,7.733378,15284.333333
1,35.193972,1.385067,2021,5.94385,cropland,1634,5.549156,61.341667,12.340887,-1.442346,...,0.317,0.238625,0.19193,0.208251,278.115705,0.081446,0.023201,3.510426,7.733378,15268.8
2,35.195589,1.384528,2021,4.4506,wooded_grassland,1626,5.39333,63.41954,13.694862,-1.242844,...,0.309575,0.20645,0.191408,0.178394,279.735847,0.077992,0.012462,6.258336,7.733378,15268.8
3,35.190468,1.378869,2021,7.10625,wooded_grassland,1652,1.854871,54.527138,8.628444,-1.174888,...,0.403735,0.277304,0.221039,0.216093,276.727989,0.066324,0.01395,4.754431,7.733378,15284.333333
4,35.187234,1.379678,2021,7.06505,wooded_grassland,1629,3.340624,22.484619,12.107599,1.348318,...,0.373843,0.288648,0.22812,0.184308,274.041426,0.073882,0.013287,5.560403,7.733378,15284.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,34.345333,0.225163,2021,10.7264,cropland,1253,1.311446,278.401245,9.27949,0.996192,...,0.4545,0.302535,0.213188,0.32082,256.226705,0.117946,0.028541,4.13259,19.002268,15093.0
2129,34.34183,0.22139,2021,13.2709,cropland,1262,3.340334,324.572754,7.82507,2.215655,...,0.457992,0.242337,0.183213,0.322395,247.206551,0.140416,0.033655,4.172176,19.002268,15089.0
2130,34.338057,0.227588,2021,8.905,cropland,1252,2.62152,36.481625,9.819545,0.669065,...,0.452025,0.258288,0.18445,0.293232,251.133856,0.160531,0.038206,4.201691,19.002268,15089.0
2131,34.345872,0.224085,2021,11.0257,cropland,1253,8.238493,282.996185,9.270283,1.455356,...,0.475538,0.3797,0.336772,0.34979,258.571467,0.099154,0.026466,3.746502,19.002268,15097.0


In [37]:
# Save the tabular data
# dftrue.to_csv(os.getcwd() + '/SOC0to2007.csv', index=False)
dataframe.to_csv(os.getcwd() + '/SOC2108to3925.csv', index=False)

In [39]:
df1 = pd.read_csv(os.getcwd() + '/SOC0to2107.csv')
df2 = pd.read_csv(os.getcwd() + '/SOC2108to3925.csv')
# df3 = pd.read_csv(os.getcwd() + '/oldCSV/SOC2551to2700.csv')
# df4 = pd.read_csv(os.getcwd() + '/oldCSV/SOC2701to3927.csv')

pd.concat([df1, df2], ignore_index=True).to_csv(os.getcwd() + '/SOC3925.csv', index=False)

In [48]:
# Extract Landsat, Sentinel-1, topography and climate data at the each site

dataframe = pd.DataFrame(columns=['longitude', 'latitude', 'year', 'LC', 'siteid', 'site', 'SOC', 
                                  'elev', 'slope', 'aspect', 'roughness', 'twi', 
                                  'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI', 'VV', 'VH', 'VV/VH', 'prec', 'temp'
                                  ])

for i in range(len(df)):
  print(i)

  for j in range(8):
    elev, slope = getElev(df.iloc[i, 0], df.iloc[i, 1])[0], getElev(df.iloc[i, 0], df.iloc[i, 1])[1]
    aspect, roughness, twi = getTopo(df.iloc[i, 0], df.iloc[i, 1])[0], getTopo(df.iloc[i, 0], df.iloc[i, 1])[1], getTopo(df.iloc[i, 0], df.iloc[i, 1])[2]
    newCol = pd.Series( df.iloc[i, :7].to_list() + [ elev[1][4], slope[1][4], aspect[1][4], roughness[1][4], twi[1][4] ], index=dataframe.columns[:12] )
    
    landsat = get_landsat(df.iloc[i, 0], df.iloc[i, 1], 
                      ee.Date(str(df.iloc[i, 2]-4) + '-07-01').advance(6*j, 'month'), 
                      ee.Date(str(df.iloc[i, 2]-4) + '-12-31').advance(6*j, 'month')
                      )
    sar = getSAR(df.iloc[i, 0], df.iloc[i, 1], 
                      ee.Date(str(df.iloc[i, 2]-4) + '-07-01').advance(6*j, 'month'), 
                      ee.Date(str(df.iloc[i, 2]-4) + '-12-31').advance(6*j, 'month')
                      )
      
    precipitation, temperature = getPrecip(df.iloc[i, 0], df.iloc[i, 1], 
                      ee.Date(str(df.iloc[i, 2]-4) + '-07-01').advance(6*j, 'month'), 
                      ee.Date(str(df.iloc[i, 2]-4) + '-12-31').advance(6*j, 'month')
                      )[0], getPrecip(df.iloc[i, 0], df.iloc[i, 1], 
                      ee.Date(str(df.iloc[i, 2]-4) + '-07-01').advance(6*j, 'month'), 
                      ee.Date(str(df.iloc[i, 2]-4) + '-12-31').advance(6*j, 'month')
                      )[1]                      
          
    addCol = pd.Series( [ landsat[1][l] for l in range(4, 11) ] 
                      + [ sar[1][l] for l in range(4, 6) ] + [ sar[1][4]/sar[1][5] ]
                      + [ precipitation[1][4], temperature[1][4] ],
                      index=dataframe.columns[12:24] 
                      )
    # newCol = newCol.append(addCol)
    newCol = pd.concat([newCol, addCol])
    # print('newCol', newCol)

    if newCol.isna().any() == True:
      print(newCol)
    
    # dataframe = dataframe.append(newRow, ignore_index=True)
    dataframe.loc[len(dataframe)] = newCol
    # print(dataframe)

  if i == 0:
    print(dataframe)

dataframe

0
   longitude   latitude  year         LC          siteid        site   
0  31.329419 -26.283403  2020  grassland  ludzeludze_4_5  ludzeludze  \
1  31.329419 -26.283403  2020  grassland  ludzeludze_4_5  ludzeludze   
2  31.329419 -26.283403  2020  grassland  ludzeludze_4_5  ludzeludze   
3  31.329419 -26.283403  2020  grassland  ludzeludze_4_5  ludzeludze   
4  31.329419 -26.283403  2020  grassland  ludzeludze_4_5  ludzeludze   
5  31.329419 -26.283403  2020  grassland  ludzeludze_4_5  ludzeludze   
6  31.329419 -26.283403  2020  grassland  ludzeludze_4_5  ludzeludze   
7  31.329419 -26.283403  2020  grassland  ludzeludze_4_5  ludzeludze   

        SOC elev      slope    aspect  ...     SR_B4     SR_B5     SR_B6   
0  18.08215  733  14.546583  6.157959  ...   0.12142  0.332593  0.264338  \
1  18.08215  733  14.546583  6.157959  ...  0.332249  0.477463  0.384169   
2  18.08215  733  14.546583  6.157959  ...  0.600718  0.646876  0.557102   
3  18.08215  733  14.546583  6.157959  ...  0

In [ ]:
# Extract Landsat, Sentinel-1, topography and climate data at the each site

dataframe = pd.DataFrame(columns=['longitude', 'latitude', 'year', 'LC', 'siteid', 'site', 'SOC', 
                                  'elev', 'slope', 'aspect', 'roughness', 'twi', 
                                  'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI', 'VV', 'VH', 'VV/VH', 'prec', 'temp'
                                  ])

for i in range(len(df)):
  print(i)

  elev, slope = getElev(df.iloc[i, 0], df.iloc[i, 1])[0], getElev(df.iloc[i, 0], df.iloc[i, 1])[1]
  aspect, roughness, twi = getTopo(df.iloc[i, 0], df.iloc[i, 1])[0], getTopo(df.iloc[i, 0], df.iloc[i, 1])[1], getTopo(df.iloc[i, 0], df.iloc[i, 1])[2]
  newCol = pd.Series( df.iloc[i, :7].to_list() + [ elev[1][4], slope[1][4], aspect[1][4], roughness[1][4], twi[1][4] ], index=dataframe.columns[:12] )
  
  for j in range(2):
    landsat = get_landsat(df.iloc[i, 0], df.iloc[i, 1], 
                       ee.Date(str(df.iloc[i, 2]-1) + '-01-01').advance(0, 'year').advance(1*j, 'year'), 
                       ee.Date(str(df.iloc[i, 2]-1) + '-12-31').advance(0, 'year').advance(1*j, 'year')
                       )
    sar = getSAR(df.iloc[i, 0], df.iloc[i, 1], 
                       ee.Date(str(df.iloc[i, 2]-1) + '-01-01').advance(0, 'year').advance(1*j, 'year'), 
                       ee.Date(str(df.iloc[i, 2]-1) + '-12-31').advance(0, 'year').advance(1*j, 'year')
                       )
    
    precipitation, temperature = getPrecip(df.iloc[i, 0], df.iloc[i, 1], 
                       ee.Date(str(df.iloc[i, 2]-1) + '-01-01').advance(0, 'year').advance(1*j, 'year'), 
                       ee.Date(str(df.iloc[i, 2]-1) + '-12-31').advance(0, 'year').advance(1*j, 'year')
                       )[0], getPrecip(df.iloc[i, 0], df.iloc[i, 1], 
                       ee.Date(str(df.iloc[i, 2]-1) + '-01-01').advance(0, 'year').advance(1*j, 'year'), 
                       ee.Date(str(df.iloc[i, 2]-1) + '-12-31').advance(0, 'year').advance(1*j, 'year')
                       )[1]                      
        
    addCol = pd.Series( [ landsat[1][l] for l in range(4, 12) ] 
                       + [ sar[1][l] for l in range(4, 6) ] + [ sar[1][4]/sar[1][5] ]
                       + [ precipitation[1][4], temperature[1][4] ],
                       index=dataframe.columns[10+j*13:23+j*13] 
                       )
    # newCol = newCol.append(addCol)
    newCol = pd.concat([newCol, addCol])
    # print('newCol', newCol)

    
    if newCol.isna().any() == True:
      print(newCol)
    
  # dataframe = dataframe.append(newRow, ignore_index=True)
  dataframe.loc[i] = newCol
  # print(dataframe)

dataframe